# Discover EOPF Zarr - Sentinel-2 L2A

## Introduction
This tutorial introduces you to the structure of a Zarr sample for **Sentinel-2 L2A** data. We will demonstrate how to access and open a Zarr product sample with `xarray`, how to visualise the `zarr` encoding structure, explore embedded information, and retrieve relevant metadata for further processing.

### Prerequisites
This tutorial uses a re-processed sample dataset frome the [EOPF Sentinel Zarr Samples Service STAC API](https://stac.browser.user.eopf.eodc.eu/) that is available for download [here](https://objects.eodc.eu/e05ab01a9d56408d82ac32d69a5aae2a:202506-s02msil2a/10/products/cpm_v256/S2C_MSIL2A_20250610T103641_N0511_R008_T32UMD_20250610T132001.zarr). 

The selected `zarr` product is a Sentinel-2 L2A tile from the 10th of June 2025 (file name: `S2C_MSIL2A_20250610T103641_N0511_R008_T32UMD_20250610T132001.zarr.`).

<hr>

#### Import libraries

In [1]:
import os
import xarray as xr

#### Helper functions

##### `print_gen_structure`
This function helps us to retrieve an visualise the names for each of the stored groups inside a `zarr`. As an output, it will print a general overview of elements inside the `zarr`.

In [2]:
def print_gen_structure(node, indent=""):
    print(f"{indent}{node.name}")     #allows us access each node
    for child_name, child_node in node.children.items(): #loops inside the selected nodes to extract naming
        print_gen_structure(child_node, indent + "  ") # prints the name of the selected nodes

<hr>

## Open a Zarr Store

In a first step, we use the function `open_datatree()` from the `xarray` library to open a Zarr store as a DataTree.<br>
Inside, we ned to define the following key word arguments:<br>

- `filename_or_obj`: path leading to a `zarr` store
- `engine`: `'eopf-zarr'`, designed for the EOPF `zarr` by ESA. 
- `op_mode`: extension by the `xarray-eopf` development for allowing an analysis or native mode. For more information visit the [xarray-eopf](https://eopf-sample-service.github.io/xarray-eopf/) documentation.
- `chunks`: loads the data with dask using the engine’s preferred chunk size, generally identical to the format’s chunk size

The final print of the `DataTree` object is commented out, as the display can be quite extensive, showing the entire content within the Zarr. An alternative is to apply a helper function that only displays the higher level structure as shown in the next code cell.

In [3]:
s2l2a_zarr_sample= xr.open_datatree('https://objects.eodc.eu/e05ab01a9d56408d82ac32d69a5aae2a:202506-s02msil2a/10/products/cpm_v256/S2C_MSIL2A_20250610T103641_N0511_R008_T32UMD_20250610T132001.zarr',
    engine="eopf-zarr", # storage format
    op_mode="native", # no analysis mode
    chunks={}, # allows to open the default chunking
)

If we apply the helper function `print_gen_structure` on the root of the DataTree object, we will get a listing of the tree-like structure of the object. We can see all Zarr groups, such as `measurements`, `quality` and `conditions`, their sub-groups and content.

In [4]:
print("Zarr Sentinel 2 L2A Structure")
print_gen_structure(s2l2a_zarr_sample.root) 
print("-" * 30)

Zarr Sentinel 2 L2A Structure
None
  conditions
    geometry
    mask
      detector_footprint
        r10m
        r20m
        r60m
      l1c_classification
        r60m
      l2a_classification
        r20m
        r60m
    meteorology
      cams
      ecmwf
  measurements
    reflectance
      r10m
      r20m
      r60m
  quality
    atmosphere
      r10m
      r20m
      r60m
    l2a_quicklook
      r10m
      r20m
      r60m
    mask
      r10m
      r20m
      r60m
    probability
      r20m
------------------------------


## Extract information from Zarr groups

In a next step, we can explore the content of individual Zarr groups. By specifying the name of the group and subgroup and adding it into square brackets, we can extract the content of the relevant group. Let us for example extract the content of the subgroup `reflectance` under `measurements`.

As a result, it is visible that there are three subgroups of the parent node `measurements/reflectance`: `r10`, `r20` and `r60`, which are the DataArrays with the three different resolutions of the Sentinel-2 L2A data.

The `xarray.DataTree` structure allows the exploration of additional group-related metadata and information. For example, we can find the `chunksize` of each array and the coordinates.

In [5]:
# Retrieving the reflectance groups:
s2l2a_zarr_sample["measurements/reflectance"]

<xarray.DataTree 'reflectance'>
Group: /measurements/reflectance
├── Group: /measurements/reflectance/r10m
│       Dimensions:  (y: 10980, x: 10980)
│       Coordinates:
│         * x        (x) int64 88kB 399965 399975 399985 399995 ... 509735 509745 509755
│         * y        (y) int64 88kB 5900035 5900025 5900015 ... 5790265 5790255 5790245
│       Data variables:
│           b02      (y, x) float64 964MB dask.array<chunksize=(1830, 1830), meta=np.ndarray>
│           b03      (y, x) float64 964MB dask.array<chunksize=(1830, 1830), meta=np.ndarray>
│           b04      (y, x) float64 964MB dask.array<chunksize=(1830, 1830), meta=np.ndarray>
│           b08      (y, x) float64 964MB dask.array<chunksize=(1830, 1830), meta=np.ndarray>
├── Group: /measurements/reflectance/r20m
│       Dimensions:  (y: 5490, x: 5490)
│       Coordinates:
│         * x        (x) int64 44kB 399970 399990 400010 400030 ... 509710 509730 509750
│         * y        (y) int64 44kB 5900030 5900010 5899990 ... 5790290 5790270 5790250
│       Data variables:
│           b01      (y, x) float64 241MB dask.array<chunksize=(915, 915), meta=np.ndarray>
│           b02      (y, x) float64 241MB dask.array<chunksize=(915, 915), meta=np.ndarray>
│           b03      (y, x) float64 241MB dask.array<chunksize=(915, 915), meta=np.ndarray>
│           b04      (y, x) float64 241MB dask.array<chunksize=(915, 915), meta=np.ndarray>
│           b05      (y, x) float64 241MB dask.array<chunksize=(915, 915), meta=np.ndarray>
│           b06      (y, x) float64 241MB dask.array<chunksize=(915, 915), meta=np.ndarray>
│           b07      (y, x) float64 241MB dask.array<chunksize=(915, 915), meta=np.ndarray>
│           b11      (y, x) float64 241MB dask.array<chunksize=(915, 915), meta=np.ndarray>
│           b12      (y, x) float64 241MB dask.array<chunksize=(915, 915), meta=np.ndarray>
│           b8a      (y, x) float64 241MB dask.array<chunksize=(915, 915), meta=np.ndarray>
└── Group: /measurements/reflectance/r60m
        Dimensions:  (y: 1830, x: 1830)
        Coordinates:
          * x        (x) int64 15kB 399990 400050 400110 400170 ... 509610 509670 509730
          * y        (y) int64 15kB 5900010 5899950 5899890 ... 5790390 5790330 5790270
        Data variables:
            b01      (y, x) float64 27MB dask.array<chunksize=(305, 305), meta=np.ndarray>
            b02      (y, x) float64 27MB dask.array<chunksize=(305, 305), meta=np.ndarray>
            b03      (y, x) float64 27MB dask.array<chunksize=(305, 305), meta=np.ndarray>
            b04      (y, x) float64 27MB dask.array<chunksize=(305, 305), meta=np.ndarray>
            b05      (y, x) float64 27MB dask.array<chunksize=(305, 305), meta=np.ndarray>
            b06      (y, x) float64 27MB dask.array<chunksize=(305, 305), meta=np.ndarray>
            b07      (y, x) float64 27MB dask.array<chunksize=(305, 305), meta=np.ndarray>
            b09      (y, x) float64 27MB dask.array<chunksize=(305, 305), meta=np.ndarray>
            b11      (y, x) float64 27MB dask.array<chunksize=(305, 305), meta=np.ndarray>
            b12      (y, x) float64 27MB dask.array<chunksize=(305, 305), meta=np.ndarray>
            b8a      (y, x) float64 27MB dask.array<chunksize=(305, 305), meta=np.ndarray>

## Extract Zarr metadata on different levels

Through `s2l2a_zarr_sample.attrs[]` we are able to visualise both the `stac_discovery` and `other_metadata` included in the `zarr` store. <br>
<br>
For the properties inside `stac_discovery` for example we can get the parameters included:

In [ ]:
# STAC metadata style:
print(list(s2l2a_zarr_sample.attrs["stac_discovery"].keys()))

['assets', 'bbox', 'geometry', 'id', 'links', 'properties', 'stac_extensions', 'stac_version', 'type']


And from `other_metadata`, we are able to retrieve the information specific to the instrument variables.

In [28]:
# Complementing metadata:
print(list(s2l2a_zarr_sample.attrs["other_metadata"].keys()))

['AOT_retrieval_model', 'L0_ancillary_data_quality', 'L0_ephemeris_data_quality', 'NUC_table_ID', 'SWIR_rearrangement_flag', 'UTM_zone_identification', 'absolute_location_assessment_from_AOCS', 'band_description', 'declared_accuracy_of_AOT_model', 'declared_accuracy_of_radiative_transfer_model', 'declared_accuracy_of_water_vapour_model', 'electronic_crosstalk_correction_flag', 'eopf_category', 'geometric_refinement', 'history', 'horizontal_CRS_code', 'horizontal_CRS_name', 'mean_sensing_time', 'mean_sun_azimuth_angle_in_deg_for_all_bands_all_detectors', 'mean_sun_zenith_angle_in_deg_for_all_bands_all_detectors', 'mean_value_of_aerosol_optical_thickness', 'mean_value_of_total_water_vapour_content', 'meteo', 'multispectral_registration_assessment', 'onboard_compression_flag', 'onboard_equalization_flag', 'optical_crosstalk_correction_flag', 'ozone_source', 'ozone_value', 'percentage_of_degraded_MSI_data', 'planimetric_stability_assessment_from_AOCS', 'product_quality_status', 'reflectanc

We are also, able to visualise the metadata included at 

## Now it is your turn

As we are able to retrieve several items from the [EOPF Sentinel Zarr Samples Service STAC API](https://stac.browser.user.eopf.eodc.eu/), lets try the following:
<br>
1. Go to the [Sentinel-2 Level-2A collection](https://stac.browser.user.eopf.eodc.eu/collections/sentinel-2-l2a)
2. Choose an item of interest.
3. Replicate the workflow and explore the item's `datetime`. When was it retrieved?
4. What are the dimensions?
5. What is the detailed location of the item?

## Conclusion
This tutorial provides an initial understanding of the `zarr` structure for a Sentinel-2 L2A sample.

By using the `xarray` library, we can effectively navigate and inspect the different components within the `zarr` format, including its metadata and array organisation. 
This foundation will help deeply undestand the subsequent data analysis and processing workflows intended in our EOPF 101 series.

In [ ]:
s2l2a_zarr_sample[]

{'other_metadata': {'AOT_retrieval_model': 'CAMS',
  'L0_ancillary_data_quality': '2',
  'L0_ephemeris_data_quality': '3',
  'NUC_table_ID': 2,
  'SWIR_rearrangement_flag': 'null',
  'UTM_zone_identification': 'S2C_OPER_MSI_L2A_TL_2CPS_20250610T132001_A003982_T32UMD_N05.11',
  'absolute_location_assessment_from_AOCS': '\n          ',
  'band_description': {'01': {'bandwidth': '20.0',
    'central_wavelength': 444.2,
    'onboard_compression_rate': '2.655',
    'onboard_integration_time': '6.5711904',
    'physical_gain': '4.71697486',
    'spectral_response_step': '1',
    'spectral_response_values': '0.00182758 0.00176219 0.00174835 0.00177315 0.00174166 0.00175671 0.00171909 0.00172996 0.00170544 0.00163243 0.00156834 0.00156899 0.00149403 0.00151818 0.00155589 0.00159342 0.0017176 0.00239482 0.01343324 0.16074943 0.56856624 0.79337977 0.83889418 0.8692385 0.88238687 0.90595802 0.92322662 0.91765842 0.90681059 0.89970444 0.90902188 0.91999798 0.93381186 0.95270212 0.97465774 0.991290

In [40]:
print(list(s2l2a_zarr_sample.attrs.keys()))

['other_metadata', 'stac_discovery']


In [48]:
s2l2a_zarr_sample2= xr.open_dataset('https://objects.eodc.eu/e05ab01a9d56408d82ac32d69a5aae2a:202506-s02msil2a/10/products/cpm_v256/S2C_MSIL2A_20250610T103641_N0511_R008_T32UMD_20250610T132001.zarr',
    engine="eopf-zarr", # storage format
    op_mode="native", # no analysis mode
    chunks={}, # allows to open the default chunking
)

In [49]:
s2l2a_zarr_sample2

<xarray.Dataset> Size: 9GB
Dimensions:                                            (
                                                        conditions_geometry_angle: 2,
                                                        conditions_geometry_band: 13,
                                                        conditions_geometry_y: 23,
                                                        conditions_geometry_x: 23,
                                                        conditions_geometry_detector: 7,
                                                        ...
                                                        quality_mask_r20m_y: 5490,
                                                        quality_mask_r20m_x: 5490,
                                                        quality_mask_r60m_y: 1830,
                                                        quality_mask_r60m_x: 1830,
                                                        quality_probability_y: 5490,
                                                        quality_probability_x: 5490)
Coordinates: (12/63)
  * conditions_geometry_angle                          (conditions_geometry_angle) <U7 56B ...
  * conditions_geometry_band                           (conditions_geometry_band) <U3 156B ...
  * conditions_geometry_detector                       (conditions_geometry_detector) int64 56B ...
  * conditions_geometry_x                              (conditions_geometry_x) int64 184B ...
  * conditions_geometry_y                              (conditions_geometry_y) int64 184B ...
  * conditions_mask_detector_footprint_r10m_x          (conditions_mask_detector_footprint_r10m_x) int64 88kB ...
    ...                                                 ...
  * quality_mask_r20m_y                                (quality_mask_r20m_y) int64 44kB ...
  * quality_mask_r60m_x                                (quality_mask_r60m_x) int64 15kB ...
  * quality_mask_r60m_y                                (quality_mask_r60m_y) int64 15kB ...
    quality_probability_band                           int64 8B ...
  * quality_probability_x                              (quality_probability_x) int64 44kB ...
  * quality_probability_y                              (quality_probability_y) int64 44kB ...
Data variables: (12/86)
    conditions_geometry_mean_sun_angles                (conditions_geometry_angle) float64 16B dask.array<chunksize=(2,), meta=np.ndarray>
    conditions_geometry_mean_viewing_incidence_angles  (conditions_geometry_band, conditions_geometry_angle) float64 208B dask.array<chunksize=(13, 2), meta=np.ndarray>
    conditions_geometry_sun_angles                     (conditions_geometry_angle, conditions_geometry_y, conditions_geometry_x) float64 8kB dask.array<chunksize=(2, 23, 23), meta=np.ndarray>
    conditions_geometry_viewing_incidence_angles       (conditions_geometry_band, conditions_geometry_detector, conditions_geometry_angle, conditions_geometry_y, conditions_geometry_x) float64 770kB dask.array<chunksize=(7, 4, 2, 23, 23), meta=np.ndarray>
    conditions_mask_detector_footprint_r10m_b02        (conditions_mask_detector_footprint_r10m_y, conditions_mask_detector_footprint_r10m_x) uint8 121MB dask.array<chunksize=(1830, 1830), meta=np.ndarray>
    conditions_mask_detector_footprint_r10m_b03        (conditions_mask_detector_footprint_r10m_y, conditions_mask_detector_footprint_r10m_x) uint8 121MB dask.array<chunksize=(1830, 1830), meta=np.ndarray>
    ...                                                 ...
    quality_mask_r20m_b8a                              (quality_mask_r20m_y, quality_mask_r20m_x) uint8 30MB dask.array<chunksize=(915, 915), meta=np.ndarray>
    quality_mask_r60m_b01                              (quality_mask_r60m_y, quality_mask_r60m_x) uint8 3MB dask.array<chunksize=(305, 305), meta=np.ndarray>
    quality_mask_r60m_b09                              (quality_mask_r60m_y, quality_mask_r60m_x) uint8 3MB dask.array<chunksize=(305, 305), meta=np.ndar